# Assignment 2
#### 02806 - Social data analysis and visualization 

_____
_____
_____
_____
_____

## Part 1 - Questions to text and lectures

***Exercise 1.1***

*What is the Oxford English Dictionary's defintion of a narrative?*

***Answer:*** 

*“An account of a series of events, facts, etc., given in order and with the establishing of connections between them.”*

____

***Exercise 1.2***

*What is your favorite visualization among the examples in section 3? Explain why in a few words.*

***Answer:***

*Our favorite is 3.2 - 'Budget Forecasts'. The idea of single-frame interactivity is excellent, since it compacts the visualisation, while still given a great deal of information. It also has a nice tacit tutorial, allowing users to participate, as if they are a part of exploring the visualisation. The added mouse-over (tooltip) details augments this compact graph with enough information - when you need it.*

_____

***Exercise 1.3***

*What's the point of Figure 7?*

***Answer:*** *The point with the figure is that a lot of different genres of presentation all have som ordering of the data element, and an interactive element of the data visualisation. However, most of them actually fail in (3) giving the user a tutorial on how to use it, or (4) don't use enough messaging to clearly narrate the story*
___

***Exercise 1.4***

*Use Figure 7 to find the most common design choice within each category for the Visual narrative and Narrative structure (the categories within visual narrative are 'visual structuring', 'highlighting', etc).*

***Answer:***

*The most common design choices in each category is:*

- **Visual narrative**
    - *Visual structuring*
        - *Consistent Visual Platform*
    - *Highlighting*
        - *Feature distinction*
    - *Transition guidance*
        - *Object continuity*
- **Narrative structure***
    - *Ordering*
        - *User directed path*
    - *Interactivity*
        - *Filtering / Selection / Search*
    - *Messaging*
        - *Captions / Headlines*
        
_____

***Exercise 1.5*** 

*Check out Figure 8 and section 4.3. What is your favorite genre of narrative visualization? Why? What is your least favorite genre? Why?*

***Answer:***

*Our favorite genre is Partitioned Poster. A poster can carry a wealth of information, including charts and legends, and can partition the information and present it intuitively. The vivid illustrations are full of fun, and the process of getting the details from the above is very inspiring. My least favorite is Flow Chart for it is boring and unintuitive. Furthermore, flow chart can't reveal changes in time. It will inculcate readers too strongly in the logic of the matter which may fail to combine with some details.*

_____

***Exercise 1.6***

*What are the three key elements to keep in mind when you design an explanatory visualization?*

***Answer:*** *There are 3 key elements:*

- *Start with the question (What is the question that you want to answer and communicate?)*
- *Allow exploration (Let the users investigate the data. Therefore something like D3, or even Tableau can be used)*
- *Know you readers (Make sure that you are not dumbing something down for instance to peers. Or making it overly complicated when it needs to be cross-disciplinary)*
____



***Exercise 1.7***

*In the video I talk about (1) overview first, (2) zoom and filter, (3) details on demand.*
- *Go online and find a visualization that follows these principles (don't use one from the video).*
- *Explain how it does achieves (1)-(3). It might be useful to use screenshots to illustrate your explanation.*

***Answer:***
- *Online we found [this link](http://manpopex.us/)*
    - *(1) First you get a good overview of dynamic population of Manhattan.* 
    - *(2) Then you can zoom in and filter the data that you need, for instance, we can select one district, instead of the 12 districts highlighted.*
    - *(3) Then we can specifically select data that happens on each day, to see how much it changes over the day*
_________

***Exercise 1.8***

*Explain in your own words: How is explanatory data analysis different from exploratory data analysis?*

***Answer:*** *Exploratory data analysis is where you are looking for correlations and interesting stories that the data has to tell you. This is opposed to the explanatory data analysis, where you've found the interesting story that the data has to show, and you've spent time on visualising that story, so that others may see it as well.*
_____

## Part 2: Random forest and weather

The aim here is to recreate the work you did in Part 1-3 of the Week 7 lecture. I've phrased things differently relative to the exercise to make the purpose more clear.

This exercise is divided into two parts:

- **Part 2A** - Random forest binary classification
- **Part 2B** - Info from weather features


### Part 2A: Random forest binary classification

* Using the and instructions and material from Week 7, build a *random forest* classifier to distinguish between two types (you choose) of crime using on spatio-temporal (where/when) features of data describing the two crimes. When you're done, you should be able to give the classifier a place and a time, and it should tell you which of the two  types of crime happened there.
  - Explain about your choices for training/test data, features, and encoding. (You decide how to present your results, but here are some example topics to consider: Did you balance the training data? What are the pros/cons of balancing? Do you think your model is overfitting? Did you choose to do cross-validation? Which specific features did you end up using? Why? Which features (if any) did you one-hot encode? Why ... or why not?))
  - Report accuracy. Discuss the model performance.

***We start off by importing pandas, numpy, and the policing dataset***

In [2]:
import numpy as np
import pandas as pd

df_all = pd.read_csv('../../Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv')

***We do some filtering of the data, so that we only have the `CATEGORY`, `DAYOFWEEK`, `TIME`, `PDDISTRICT`, AND `MONTH`***

In [4]:
# We chose the columns of interest
df_all = df_all.loc[:, ["Category","DayOfWeek","Date","Time","PdDistrict"]]

# We create a dataset for prostitution
df_pro = df_all[df_all['Category'] == 'PROSTITUTION']
# We create a dataset for vandalism
df_van = df_all[df_all['Category'] == 'VANDALISM']
# We sample 15000 data entries from each dataset to create a balanced dataset
df_pro = df_pro.sample(15000)
df_van = df_van.sample(15000)
# We concat the datasets
df = pd.concat([df_pro,df_van])

# We convert the hour to just be the hour, without the minutes (i.e 22:23 = 22:00)
# and also convert it to the appropriate timezone to match the weather data later
df["Date"] = df.apply(lambda x: pd.to_datetime(x.Date + " " + x.Time).round("H").tz_localize("ETC/GMT-7"), axis = 1)

# We convert the day of the week into an integer
df['DayOfWeek'] = df['DayOfWeek'].map({'Monday':0,'Tuesday':1,'Wednesday':2,'Thursday':3,'Friday':4,'Saturday':5,'Sunday':6})

# We convert the district into an integer 
df['PdDistrict'] = df['PdDistrict'].map({'BAYVIEW':0,'CENTRAL':1,'INGLESIDE':2,'MISSION':3,'NORTHERN':4,'PARK':5,'RICHMOND':6,'SOUTHERN':7,'TARAVAL':8,'TENDERLOIN':9})

# We extract the month from the time data
df['Month'] = pd.to_datetime(df['Date']).dt.month

# We extract the hour from the time data
df['Time'] = pd.to_datetime(df['Time']).dt.hour

# We make the categorical data for prostitution and vandalism into integers
df['Category'] = df['Category'].map({'PROSTITUTION':0,'VANDALISM':1})

# We set the index to the datetime
df.set_index('Date', inplace=True)

# And display the first 10 incidents in the dataframe
df.head(10)

,Category,DayOfWeek,Time,PdDistrict,Month
Date,,,,,
2017-04-29 03:00:00+07:00,0,5,3,3,4
2013-01-03 00:00:00+07:00,0,2,23,3,1
2008-06-13 22:00:00+07:00,0,4,22,4,6
2004-05-18 18:00:00+07:00,0,1,17,7,5
2008-04-23 05:00:00+07:00,0,2,4,1,4
2014-11-25 18:00:00+07:00,0,1,18,1,11
2009-09-03 20:00:00+07:00,0,3,19,4,9
2013-03-12 19:00:00+07:00,0,1,19,9,3
2009-04-20 18:00:00+07:00,0,0,18,3,4


***A training and test dataset is chosen with the use of the sklearn.model_selection package***

In [6]:
# We choose our features
X = df.iloc[:, 1:5].values
# We choose targets
y = df.iloc[:, 0].values

from sklearn.model_selection import train_test_split

# We split it in training and test data with our features and target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=20)

***Now a fit is made on the dataset with a random forest classifier***

In [8]:
from sklearn.ensemble import RandomForestClassifier

# Initialise the randomforestclassifier as classifier, with 20 estimators
classifier = RandomForestClassifier(n_estimators=20, random_state=20)
# We fit the classifier on our training set with features and target
classifier.fit(X_train, y_train)

# We make a prediction on the test set 
y_pred = classifier.predict(X_test)

# We a prediction on the training set
y_train_pred = classifier.predict(X_train)

***Now the training and test set accuracy can be reported as follows:***

In [10]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#print(confusion_matrix(y_test,y_pred))
#print(classification_report(y_test,y_pred))
print('training set accuracy:')
print(accuracy_score(y_train, y_train_pred))
print('test set accuracy:')
print(accuracy_score(y_test, y_pred))

training set accuracy:
0.8892962962962963
test set accuracy:
0.809


*From the incident reports dataset, the two categories `PROSTITUTION` and `VANDALISM` was chosen. The training and test set was divided at a ratio of 9:1 with no validation set. In order to insure a balanced dataset, 15000 data points were sampled from both datasets. Using a balanced dataset should greatly increase the model prediction accuracy in the future, as it is not skewed by an uneven amount of data. For instance, if we had 15000 datapoints with the `PROSTITUTION` category and 1000 points of the `VANDALISM` category, then the model is much more likely to predict `PROSTITUTION` over `VANDLISM`*

In the above model, it is clear that the training set prediction accuracy is 89% compared to the test set prediction accuracy of 82%. This is of course not a massive descrepancy, but it should be taken into consideration, if the model prediction accuracy should be improved - ideally it will give the same accuracy for the training and test set. The small difference in prediction accuracy may be accredited to the fact that the Random Forest has been used over just a Decision Tree - this is a good way of eliminating the risk of overfitting on datasets, since it will generate several decision trees and then use the model that overfits the least but with the higest prediction accuracy possible. 

In this case, one-hot encoding was not used - categorical data was transformed into integers to allow for easier interpretation of the classifier.

### Part 2B: Info from weather features

* Now add features from weather data to your random forest. 
  - Report accuracy. 
  - Discuss how the model performance changes relative to the version with no weather data.
  - Discuss what you have learned about crime from including weather data in your model.

In [12]:
weather = pd.read_csv("weather.csv", parse_dates=["date"],date_parser=lambda x: pd.to_datetime(x).tz_convert(None).tz_localize("Etc/GMT+3").tz_convert("Etc/GMT-7"))
weather['Date'] = weather['date']
weather = weather.drop('date', 1)
weather.set_index('Date', inplace=True)
weather['weather'] = pd.Categorical(weather['weather'], categories=weather['weather'].unique()).codes
weather.head()

,temperature,humidity,weather,wind_speed,wind_direction,pressure
Date,,,,,,
2012-10-01 23:00:00+07:00,16.330000,88.0,0,2.0,150.0,1009.0
2012-10-02 00:00:00+07:00,16.324993,87.0,1,2.0,147.0,1009.0
2012-10-02 01:00:00+07:00,16.310618,86.0,1,2.0,141.0,1009.0
2012-10-02 02:00:00+07:00,16.296243,85.0,1,2.0,135.0,1009.0
2012-10-02 03:00:00+07:00,16.281869,84.0,1,2.0,129.0,1009.0


In [14]:
df_merge = pd.merge(df, weather, how='inner', left_index=True, right_index=True).dropna()
df_merge.head()

,Category,DayOfWeek,Time,PdDistrict,Month,temperature,humidity,weather,wind_speed,wind_direction,pressure
Date,,,,,,,,,,,
2012-10-02 02:00:00+07:00,0,1,2,3,10,16.296243,85.0,1,2.0,135.0,1009.0
2012-10-02 02:00:00+07:00,0,1,1,3,10,16.296243,85.0,1,2.0,135.0,1009.0
2012-10-02 02:00:00+07:00,0,1,2,3,10,16.296243,85.0,1,2.0,135.0,1009.0
2012-10-02 15:00:00+07:00,1,1,15,2,10,16.109372,72.0,1,0.0,55.0,1012.0
2012-10-03 09:00:00+07:00,1,2,8,2,10,31.710000,11.0,1,7.0,280.0,1013.0


In [16]:
X = df_merge.iloc[:, 1:11].values
y = df_merge.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=20)

In [18]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [20]:
classifier = RandomForestClassifier(n_estimators=20, random_state=20)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [22]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[213  50]
 [ 27 489]]
              precision    recall  f1-score   support

           0       0.89      0.81      0.85       263
           1       0.91      0.95      0.93       516

    accuracy                           0.90       779
   macro avg       0.90      0.88      0.89       779
weighted avg       0.90      0.90      0.90       779

0.9011553273427471


(Some analysis for model including weather data...)

## Part 3: Data visualization

* Create the Bokeh visualization from Part 2 of the Week 8 Lecture, displayed in a beautiful `.gif` below. 
* Provide nice comments for your code. Don't just use the `# inline comments`, but the full Notebook markdown capabilities and explain what you're doing.

![Movie](https://github.com/suneman/socialdataanalysis2020/blob/master/files/week8_1.gif?raw=true "movie")

*Here is our gif(It is saved on the google dirve):*


![Movie](https://drive.google.com/uc?export=view&id=1maZubl3ac2JH0Z_UEkiaqWywdRQldkqF "movie")

In [24]:
df = pd.read_csv('../../Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv', parse_dates=[['Date', 'Time']])
df = df[(df['Date_Time'].dt.year >= 2010) & (df['Date_Time'].dt.year <= 2018)]

In [26]:
focuscrimes = ['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT']

df_norm = pd.DataFrame(np.zeros((24, len(focuscrimes))),columns=focuscrimes)
df_norm['Hour'] = [x for x in range(1, 25)]
# df_norm = df_norm.set_index('Hour')

for i, crime_type in enumerate(focuscrimes):
    df_CT = df[df['Category'] == crime_type]
    countByHour_CT = df_CT.groupby(df['Date_Time'].dt.hour).count()["PdId"]
    df_norm[crime_type] = [float(i)/sum(countByHour_CT.values) for i in countByHour_CT.values]
df_norm.head()

,WEAPON LAWS,PROSTITUTION,DRIVING UNDER THE INFLUENCE,ROBBERY,BURGLARY,ASSAULT,DRUNKENNESS,DRUG/NARCOTIC,TRESPASS,LARCENY/THEFT,VANDALISM,VEHICLE THEFT,STOLEN PROPERTY,DISORDERLY CONDUCT,Hour
0,0.054658,0.129965,0.120321,0.055337,0.040437,0.056427,0.080726,0.034677,0.028392,0.039461,0.054485,0.035958,0.044469,0.051670,1
1,0.039489,0.096167,0.112002,0.060035,0.027947,0.048098,0.076938,0.020679,0.020475,0.025276,0.038185,0.024205,0.033556,0.038017,2
2,0.033965,0.060976,0.098930,0.060345,0.031685,0.043706,0.070361,0.016767,0.025571,0.015452,0.035545,0.018258,0.029464,0.032346,3
3,0.023743,0.036585,0.047534,0.037375,0.033482,0.023417,0.026909,0.012391,0.021294,0.009955,0.025907,0.011993,0.023462,0.018273,4
4,0.016241,0.019512,0.017231,0.023489,0.029889,0.014566,0.014152,0.009198,0.016198,0.006559,0.017711,0.010077,0.020052,0.015123,5


In [28]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [30]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange, Legend

source = ColumnDataSource(df_norm)
p = figure(plot_width=1000, plot_height=400, x_range = FactorRange(factors=list(map(str,df_norm['Hour'].values.tolist()))), title="Crimes By Hour", x_axis_label='Hour of The Day', y_axis_label='Relative Frequency')

X axis is 24-hour which is set by 'FactorRange', a Range of values for a categorical dimesion. Other parameters are configured to set up a Bokeh figure.

In [32]:
from bokeh.palettes import Category20

bar = {}
items = []
for indx,(i,color) in enumerate(zip(focuscrimes, Category20[14])):
    bar[i] = p.vbar(x='Hour', top=i, source=source, alpha=1, color=color, muted_alpha=0.08, muted_color=color, muted=True)
    items.append((i, [bar[i]]))
legend = Legend(items=items, location=(0,-30))

To construct a bar plot, both x-axis and y-axis is selected from the Dataframe. 'alpha' decides the color depth. 'muted_color' and 'muted_alpha' are set for the interactive mode is 'Muted'.

In [34]:
p.add_layout(legend, "right")
p.legend.click_policy="mute"
show(p)

Legend is configured separately and can be clicked to mute specific bar.